In [0]:
!pip install openpyxl

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
train_path = '/Volumes/hackathon/default/dataset/train_synthetic_4000.xlsx'
test_path  = '/Volumes/hackathon/default/dataset/test_monthday.xlsx'

In [0]:
train_df = pd.read_excel(train_path)
test_df  = pd.read_excel(test_path)

In [0]:
train_df.head()

,QuoteDate,ProductCategory,ProductPrice,QuoteAmount,QuoteStatus,ConvertedToSale,Discount,Channel,CustomerAge,Region,PreQuoteVisits,PaymentMethod,day,month
0,2025-04-25,Books,45.9,41.31,Lost,0,0.10,Referral,35,Illinois,5,Credit Card,Friday,April
1,2025-05-29,Outdoor,95.0,85.50,Won,1,0.10,App,60,Texas,4,Credit Card,Thursday,May
2,2025-04-18,Beauty,56.0,50.40,Lost,0,0.10,Web,40,Georgia,4,Gift Card,Friday,April
3,2025-04-20,Outdoor,88.9,75.57,Won,1,0.15,Referral,30,Texas,3,Gift Card,Sunday,April
4,2025-04-20,Outdoor,88.9,75.57,Won,1,0.15,Referral,30,Texas,3,Gift Card,Sunday,April


In [0]:
print(train_df.shape)
print(test_df.shape)

(4000, 14)
(1000, 14)


In [0]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   QuoteDate        4000 non-null   datetime64[ns]
 1   ProductCategory  4000 non-null   object        
 2   ProductPrice     4000 non-null   float64       
 3   QuoteAmount      4000 non-null   float64       
 4   QuoteStatus      4000 non-null   object        
 5   ConvertedToSale  4000 non-null   int64         
 6   Discount         4000 non-null   float64       
 7   Channel          4000 non-null   object        
 8   CustomerAge      4000 non-null   int64         
 9   Region           4000 non-null   object        
 10  PreQuoteVisits   4000 non-null   int64         
 11  PaymentMethod    4000 non-null   object        
 12  day              4000 non-null   object        
 13  month            4000 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(

In [0]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   QuoteDate        1000 non-null   datetime64[ns]
 1   ProductCategory  1000 non-null   object        
 2   ProductPrice     1000 non-null   float64       
 3   QuoteAmount      1000 non-null   float64       
 4   QuoteStatus      1000 non-null   object        
 5   ConvertedToSale  1000 non-null   int64         
 6   Discount         1000 non-null   float64       
 7   Channel          1000 non-null   object        
 8   CustomerAge      999 non-null    float64       
 9   Region           999 non-null    object        
 10  PreQuoteVisits   999 non-null    float64       
 11  PaymentMethod    999 non-null    object        
 12  day              1000 non-null   object        
 13  month            1000 non-null   object        
dtypes: datetime64[ns](1), float64(5), int64(1

In [0]:
test_df.dropna(inplace=True)

In [0]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 0 to 999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   QuoteDate        999 non-null    datetime64[ns]
 1   ProductCategory  999 non-null    object        
 2   ProductPrice     999 non-null    float64       
 3   QuoteAmount      999 non-null    float64       
 4   QuoteStatus      999 non-null    object        
 5   ConvertedToSale  999 non-null    int64         
 6   Discount         999 non-null    float64       
 7   Channel          999 non-null    object        
 8   CustomerAge      999 non-null    float64       
 9   Region           999 non-null    object        
 10  PreQuoteVisits   999 non-null    float64       
 11  PaymentMethod    999 non-null    object        
 12  day              999 non-null    object        
 13  month            999 non-null    object        
dtypes: datetime64[ns](1), float64(5), int64(1)

In [0]:
drop_cols = ["QuoteDate", "QuoteStatus"]
train_df = train_df.drop(columns=drop_cols)
test_df  = test_df.drop(columns=drop_cols)

target_col = "ConvertedToSale"

X_test_non_en = test_df.drop(columns=[target_col])

In [0]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Identify categorical columns
cat_cols = train_df.select_dtypes(include=["object"]).columns

# Initialize OneHotEncoder (new API)
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit and transform
train_encoded = pd.DataFrame(ohe.fit_transform(train_df[cat_cols]), index=train_df.index)
test_encoded = pd.DataFrame(ohe.transform(test_df[cat_cols]), index=test_df.index)

# Add column names
train_encoded.columns = ohe.get_feature_names_out(cat_cols)
test_encoded.columns = ohe.get_feature_names_out(cat_cols)

# Merge with non-categorical data
train_df = pd.concat([train_df.drop(columns=cat_cols), train_encoded], axis=1)
test_df = pd.concat([test_df.drop(columns=cat_cols), test_encoded], axis=1)

In [0]:
import joblib
joblib.dump(ohe, "one_hot_encoder.pkl")


['one_hot_encoder.pkl']

In [0]:
cat_cols

Index(['ProductCategory', 'Channel', 'Region', 'PaymentMethod', 'day',
       'month'],
      dtype='object')

In [0]:
target_col = "ConvertedToSale"

X_train = train_df.drop(columns=[target_col])
y_train = train_df[target_col]

X_test  = test_df.drop(columns=[target_col])
y_test  = test_df[target_col]

models

In [0]:
# random forest

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=15, n_estimators=200, random_state=42)

In [0]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7067067067067067

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.73      0.83       959
           1       0.03      0.17      0.05        40

    accuracy                           0.71       999
   macro avg       0.49      0.45      0.44       999
weighted avg       0.92      0.71      0.80       999


Confusion Matrix:
 [[699 260]
 [ 33   7]]


In [0]:
!pip install xgboost
from xgboost import XGBClassifier

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 MB 45.6 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from xgboost import XGBClassifier

In [0]:
xgb_model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, ...)

In [0]:
y_pred_xgb = xgb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("\nClassification Report:\n", classification_report(y_test, y_pred_xgb))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))

Accuracy: 0.6506506506506506

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.67      0.79       959
           1       0.03      0.25      0.05        40

    accuracy                           0.65       999
   macro avg       0.49      0.46      0.42       999
weighted avg       0.92      0.65      0.76       999


Confusion Matrix:
 [[640 319]
 [ 30  10]]


In [0]:
# Get predicted values
y_pred_xgb2 = xgb_model.predict(X_test)

# True positives (predicted 1, actual 1)
won_rows = test_df[(y_test == 1) & (y_pred_xgb == 1)]

print(won_rows)
print("Number of true positive 'Won' rows:", len(won_rows))

     ProductPrice  QuoteAmount  ...  month_March  month_May
1           89.00        75.65  ...          0.0        0.0
57          62.99        59.84  ...          0.0        1.0
86          87.45        78.71  ...          0.0        1.0
133         78.00        62.40  ...          0.0        1.0
328        210.00       178.50  ...          0.0        1.0
346         95.00        85.50  ...          0.0        1.0
408        197.00       177.30  ...          0.0        0.0
414        229.00       206.10  ...          0.0        0.0
928        210.00       178.50  ...          0.0        1.0
967        285.00       256.50  ...          0.0        0.0

[10 rows x 56 columns]
Number of true positive 'Won' rows: 10


In [0]:
# List of correct row indexes
correct_idx = [1, 57, 86, 133, 328, 346, 408, 414, 928, 967]

# Get those rows from X_test
correct_rows = X_test_non_en.loc[correct_idx]
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(correct_rows)

    ProductCategory  ProductPrice  QuoteAmount  Discount   Channel  \
1            Beauty         89.00        75.65      0.15       App   
57          Apparel         62.99        59.84      0.05  Referral   
86          Outdoor         87.45        78.71      0.10    Social   
133     Electronics         78.00        62.40      0.20  Referral   
328     Electronics        210.00       178.50      0.15    Social   
346         Outdoor         95.00        85.50      0.10       App   
408     Electronics        197.00       177.30      0.10       App   
414     Electronics        229.00       206.10      0.10       App   
928     Electronics        210.00       178.50      0.15    Social   
967            Home        285.00       256.50      0.10  Referral   

     CustomerAge      Region  PreQuoteVisits  PaymentMethod        day  month  
1           49.0    Illinois             8.0         PayPal     Friday   July  
57          34.0    Missouri             2.0  Bank Transfer  Wednesda

In [0]:
joblib.dump(xgb_model, "xgb_model.pkl")

['xgb_model.pkl']

In [0]:
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 55.2 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from lightgbm import LGBMClassifier

lgb_model = LGBMClassifier(
    n_estimators=200,
    max_depth=-1,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

lgb_model.fit(X_train, y_train)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2000, number of negative: 2000
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000805 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 597
[LightGBM] [Info] Number of data points in the train set: 4000, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

LGBMClassifier(colsample_bytree=0.8, n_estimators=200, random_state=42,
               subsample=0.8)

In [0]:
y_pred_lgb = lgb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_lgb))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lgb))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_lgb))

Accuracy: 0.6696696696696697

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.69      0.80       959
           1       0.03      0.20      0.05        40

    accuracy                           0.67       999
   macro avg       0.49      0.44      0.42       999
weighted avg       0.92      0.67      0.77       999


Confusion Matrix:
 [[661 298]
 [ 32   8]]


In [0]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(max_iter=1000, solver='liblinear')
log_model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, solver='liblinear')

In [0]:
y_pred_log = log_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("\nClassification Report:\n", classification_report(y_test, y_pred_log))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_log))

Accuracy: 0.09309309309309309

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.06      0.11       959
           1       0.04      0.88      0.07        40

    accuracy                           0.09       999
   macro avg       0.48      0.47      0.09       999
weighted avg       0.89      0.09      0.11       999


Confusion Matrix:
 [[ 58 901]
 [  5  35]]


In [0]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 119.1 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier(
    iterations=200,
    depth=6,
    learning_rate=0.1,
    eval_metric='AUC',
    random_seed=42,
    verbose=False
)

cat_model.fit(X_train, y_train)

In [0]:
y_pred_cat = cat_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_cat))
print("\nClassification Report:\n", classification_report(y_test, y_pred_cat))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_cat))

Accuracy: 0.6956956956956957

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.72      0.82       959
           1       0.03      0.23      0.06        40

    accuracy                           0.70       999
   macro avg       0.49      0.47      0.44       999
weighted avg       0.92      0.70      0.79       999


Confusion Matrix:
 [[686 273]
 [ 31   9]]


In [0]:
from sklearn.ensemble import ExtraTreesClassifier

et_model = ExtraTreesClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42
)
et_model.fit(X_train, y_train)

ExtraTreesClassifier(n_estimators=200, random_state=42)

In [0]:
y_pred_et = et_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_et))
print("\nClassification Report:\n", classification_report(y_test, y_pred_et))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_et))

Accuracy: 0.6506506506506506

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.67      0.79       959
           1       0.02      0.20      0.04        40

    accuracy                           0.65       999
   macro avg       0.49      0.43      0.42       999
weighted avg       0.92      0.65      0.76       999


Confusion Matrix:
 [[642 317]
 [ 32   8]]


In [0]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb_model = HistGradientBoostingClassifier(
    max_iter=200,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)
hgb_model.fit(X_train, y_train)

HistGradientBoostingClassifier(max_depth=6, max_iter=200, random_state=42)

In [0]:
y_pred_hgb = hgb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_hgb))
print("\nClassification Report:\n", classification_report(y_test, y_pred_hgb))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_hgb))

Accuracy: 0.6606606606606606

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.68      0.79       959
           1       0.02      0.17      0.04        40

    accuracy                           0.66       999
   macro avg       0.49      0.43      0.42       999
weighted avg       0.91      0.66      0.76       999


Confusion Matrix:
 [[653 306]
 [ 33   7]]


saving the model

In [0]:
import mlflow
import mlflow.xgboost

mlflow.set_experiment("/Volumes/hackathon/default/dataset/win_predict")

with mlflow.start_run():
    mlflow.xgboost.log_model(
        xgb_model,
        artifact_path="model",
        registered_model_name="hackathon.default.win_predictor"
    )

2025-08-15 01:02:26,269 2740 ERROR _handle_rpc_error GRPC Error received
Traceback (most recent call last):
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py", line 1859, in config
    resp = self._stub.Config(req, metadata=self.metadata())
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/grpc/_interceptor.py", line 277, in __call__
    response, ignored_call = self._with_call(
                             ^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/grpc/_interceptor.py", line 332, in _with_call
    return call.result(), call
           ^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/grpc/_channel.py", line 440, in result
    raise self
  File "/databricks/python/lib/python3.11/site-packages/grpc/_interceptor.py", line 315, in continuation
    response, call = self._thunk(new_method).with_call(
                     ^^^^^^^^^^^^^^

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6732639873189170>, line 4
      1 import mlflow
      2 import mlflow.xgboost
----> 4 mlflow.set_experiment("/Volumes/hackathon/default/dataset/win_predict")
      6 with mlflow.start_run():
      7     mlflow.xgboost.log_model(
      8         xgb_model,
      9         artifact_path="model",
     10         registered_model_name="hackathon.default.win_predictor"
     11     )

File /databricks/python/lib/python3.11/site-packages/mlflow/tracking/fluent.py:140, in set_experiment(experiment_name, experiment_id)
    132 if (experiment_name is not None and experiment_id is not None) or (
    133     experiment_name is None and experiment_id is None
    134 ):
    135     raise MlflowException(
    136         message="Must specify exactly one of: `experiment_id` or `experiment_name`.",
    137         error_code=INVALID_PARAM

In [0]:
import mlflow

mlflow.set_tracking_uri("databricks")      # track to this workspace
mlflow.set_registry_uri("databricks-uc")   # register to Unity Catalog
mlflow.set_experiment("/Volumes/hackathon/win_predict")

2025/08/15 01:04:29 INFO mlflow.tracking.fluent: Experiment with name '/Volumes/hackathon/win_predict' does not exist. Creating a new experiment.


---------------------------------------------------------------------------
RestException                             Traceback (most recent call last)
File <command-6732639873189173>, line 5
      3 mlflow.set_tracking_uri("databricks")      # track to this workspace
      4 mlflow.set_registry_uri("databricks-uc")   # register to Unity Catalog
----> 5 mlflow.set_experiment("/Volumes/hackathon/win_predict")

File /databricks/python/lib/python3.11/site-packages/mlflow/tracking/fluent.py:151, in set_experiment(experiment_name, experiment_id)
    144         _logger.info(
    145             "Experiment with name '%s' does not exist. Creating a new experiment.",
    146             experiment_name,
    147         )
    148         # NB: If two simultaneous threads or processes attempt to set the same experiment
    149         # simultaneously, a race condition may be encountered here wherein experiment creation
    150         # fails
--> 151         experiment_id = client.create_exper

In [0]:
%python
import mlflow

mlflow.set_tracking_uri("databricks")      # track to this workspace
mlflow.set_registry_uri("databricks-uc")   # register to Unity Catalog

# Create the experiment if it does not exist
experiment_name = "/Volumes/hackathon/win_predict"
if not mlflow.get_experiment_by_name(experiment_name):
    mlflow.create_experiment(experiment_name)

mlflow.set_experiment(experiment_name)

---------------------------------------------------------------------------
RestException                             Traceback (most recent call last)
File <command-6732639873189174>, line 9
      7 experiment_name = "/Volumes/hackathon/win_predict"
      8 if not mlflow.get_experiment_by_name(experiment_name):
----> 9     mlflow.create_experiment(experiment_name)
     11 mlflow.set_experiment(experiment_name)

File /databricks/python/lib/python3.11/site-packages/mlflow/tracking/fluent.py:1649, in create_experiment(name, artifact_location, tags)
   1601 def create_experiment(
   1602     name: str,
   1603     artifact_location: Optional[str] = None,
   1604     tags: Optional[Dict[str, Any]] = None,
   1605 ) -> str:
   1606     """
   1607     Create an experiment.
   1608 
   (...)
   1647         Creation timestamp: 1662004217511
   1648     """
-> 1649     return MlflowClient().create_experiment(name, artifact_location, tags)

File /databricks/python/lib/python3.11/site-packages/

In [0]:
%python
# Create the directory if it does not exist
dbutils.fs.mkdirs("/Volumes/hackathon")

2025-08-15 01:06:28,285 2740 ERROR _handle_rpc_error GRPC Error received
Traceback (most recent call last):
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py", line 1723, in _execute_and_fetch_as_iterator
    for b in generator:
  File "<frozen _collections_abc>", line 330, in __next__
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/reattach.py", line 139, in send
    if not self._has_next():
           ^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/reattach.py", line 200, in _has_next
    raise e
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/reattach.py", line 172, in _has_next
    self._current = self._call_iter(
                    ^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/reattach.py", line 297, in _call_iter
    raise e
  File "/databricks/python/lib/python3.11

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-6732639873189180>, line 2
      1 # Create the directory if it does not exist
----> 2 dbutils.fs.mkdirs("/Volumes/hackathon")

File /databricks/python_shell/lib/dbruntime/remotefshandler/RemoteFsHandler.py:52, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
     49 class ExecutionError(Exception):
     50     pass
---> 52 raise ExecutionError(str(e)) from None

ExecutionError: (org.apache.spark.SparkSecurityException) [INSUFFICIENT_PERMISSIONS] Insufficient privileges:
User does not have permission MODIFY,SELECT on any file. SQLSTATE: 42501

JVM stacktrace:
org.apache.spark.SparkSecurityException
	at com.databricks.sql.acl.Unauthorized.throwInsufficientPermissionsError(PermissionChecker.scala:360)
	at com.databricks.sql.acl.CheckPermissions$.com$databricks$sql$acl$CheckPermissions$$checkAn

In [0]:
%python
import mlflow

# Create the directory if it does not exist
dbutils.fs.mkdirs("/dbfs/FileStore/hackathon")

mlflow.set_tracking_uri("databricks")      # track to this workspace
mlflow.set_registry_uri("databricks-uc")   # register to Unity Catalog

# Create the experiment if it does not exist
experiment_name = "/dbfs/FileStore/hackathon/win_predict"
if not mlflow.get_experiment_by_name(experiment_name):
    mlflow.create_experiment(experiment_name)

mlflow.set_experiment(experiment_name)

2025-08-15 01:08:28,234 2740 ERROR _handle_rpc_error GRPC Error received
Traceback (most recent call last):
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py", line 1723, in _execute_and_fetch_as_iterator
    for b in generator:
  File "<frozen _collections_abc>", line 330, in __next__
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/reattach.py", line 139, in send
    if not self._has_next():
           ^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/reattach.py", line 200, in _has_next
    raise e
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/reattach.py", line 172, in _has_next
    self._current = self._call_iter(
                    ^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/reattach.py", line 297, in _call_iter
    raise e
  File "/databricks/python/lib/python3.11

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-6732639873189189>, line 4
      1 import mlflow
      3 # Create the directory if it does not exist
----> 4 dbutils.fs.mkdirs("/dbfs/FileStore/hackathon")
      6 mlflow.set_tracking_uri("databricks")      # track to this workspace
      7 mlflow.set_registry_uri("databricks-uc")   # register to Unity Catalog

File /databricks/python_shell/lib/dbruntime/remotefshandler/RemoteFsHandler.py:52, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
     49 class ExecutionError(Exception):
     50     pass
---> 52 raise ExecutionError(str(e)) from None

ExecutionError: (org.apache.spark.SparkSecurityException) [INSUFFICIENT_PERMISSIONS] Insufficient privileges:
User does not have permission MODIFY,SELECT on any file. SQLSTATE: 42501

JVM stacktrace:
org.apache.spark.SparkSecurityException
	at com.dat

In [0]:
# 1) Are we accidentally on Spark Connect / SQL Warehouse?
import traceback, sys
print("Spark version:", spark.version)
print("Master:", spark.conf.get("spark.master"))
# 2) What URIs did MLflow pick up?
print("Tracking URI:", mlflow.get_tracking_uri())
print("Registry URI:", mlflow.get_registry_uri())
# 3) Do we have the UC model registry configured at the Spark level?
for k in ["spark.databricks.mlflow.trackId", "spark.mlflow.modelRegistryUri"]:
    try:
        print(k, "=", spark.conf.get(k))
    except Exception as e:
        print(k, "=", "<not set>")


Spark version: 4.0.0


2025-08-15 01:13:53,046 2740 ERROR _handle_rpc_error GRPC Error received
Traceback (most recent call last):
  File "/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py", line 1859, in config
    resp = self._stub.Config(req, metadata=self.metadata())
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/grpc/_interceptor.py", line 277, in __call__
    response, ignored_call = self._with_call(
                             ^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/grpc/_interceptor.py", line 332, in _with_call
    return call.result(), call
           ^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/grpc/_channel.py", line 440, in result
    raise self
  File "/databricks/python/lib/python3.11/site-packages/grpc/_interceptor.py", line 315, in continuation
    response, call = self._thunk(new_method).with_call(
                     ^^^^^^^^^^^^^^

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6732639873189201>, line 4
      2 import traceback, sys
      3 print("Spark version:", spark.version)
----> 4 print("Master:", spark.conf.get("spark.master"))
      5 # 2) What URIs did MLflow pick up?
      6 print("Tracking URI:", mlflow.get_tracking_uri())

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/conf.py:66, in RuntimeConf.get(self, key, default)
     62     op_get_with_default = proto.ConfigRequest.GetWithDefault(
     63         pairs=[proto.KeyValue(key=key, value=cast(Optional[str], default))]
     64     )
     65     operation = proto.ConfigRequest.Operation(get_with_default=op_get_with_default)
---> 66 result = self._client.config(operation)
     67 return result.pairs[0][1]

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1864, in SparkCon

saving model as pickel file and saving predictions too

In [0]:
import joblib

# Train your model (example)
# xgb_model = XGBClassifier().fit(X_train, y_train)

# Save to a file
joblib.dump(xgb_model, "xgb_model.pkl")


['xgb_model.pkl']

In [0]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 56 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ProductPrice                 4000 non-null   float64
 1   QuoteAmount                  4000 non-null   float64
 2   ConvertedToSale              4000 non-null   int64  
 3   Discount                     4000 non-null   float64
 4   CustomerAge                  4000 non-null   int64  
 5   PreQuoteVisits               4000 non-null   int64  
 6   ProductCategory_Apparel      4000 non-null   float64
 7   ProductCategory_Beauty       4000 non-null   float64
 8   ProductCategory_Books        4000 non-null   float64
 9   ProductCategory_Electronics  4000 non-null   float64
 10  ProductCategory_Home         4000 non-null   float64
 11  ProductCategory_Outdoor      4000 non-null   float64
 12  Channel_App                  4000 non-null   float64
 13  Channel_Email     

In [0]:
import joblib
model = joblib.load("xgb_model.pkl")

try:
    preds = model.predict_proba(test_df['ProductCategory', 'ProductPrice', 'QuoteAmount', 'Discount', 'Channel', 
'CustomerAge', 'Region', 'PreQuoteVisits', 'PaymentMethod', 'day', 'month'])
    print("It works with raw features — pipeline detected.")
except Exception as e:
    print("Error:", e)

Error: ('ProductCategory', 'ProductPrice', 'QuoteAmount', 'Discount', 'Channel', 'CustomerAge', 'Region', 'PreQuoteVisits', 'PaymentMethod', 'day', 'month')


In [0]:
import joblib
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Load model
model = joblib.load("xgb_model.pkl")

# Load training encoder (you must have saved it earlier)
ohe = joblib.load("one_hot_encoder.pkl")  # If you didn't save it, we can rebuild with train_df

# Load new data
new_df = pd.read_excel("/Volumes/hackathon/default/dataset/test_monthday.xlsx")

# --- Drop unwanted columns if they exist ---
cols_to_drop = ["QuoteDate", "QuoteStatus", "ConvertedToSale"]  # labels/metadata
new_df = new_df.drop(columns=[c for c in cols_to_drop if c in new_df.columns], errors="ignore")

# Identify categorical columns (same as training)
cat_cols = ["ProductCategory", "Channel", "Region", "PaymentMethod", "day", "month"]  # replace with your actual categorical cols

# Apply one-hot encoding
encoded = pd.DataFrame(ohe.transform(new_df[cat_cols]), index=new_df.index)
encoded.columns = ohe.get_feature_names_out(cat_cols)

# Merge with numeric columns

# Predict
new_df["win_probability"] = model.predict_proba(new_df_encoded)[:, 1]
new_df["ConvertedToSale_pred"] = (new_df["win_probability"] >= 0.5).astype(int)

# Save predictions
new_df.to_excel("predictions.xlsx", index=False)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File /local_disk0/.ephemeral_nfs/envs/pythonEnv-4b30f679-89b4-4961-88ab-3b049310e1f4/lib/python3.11/site-packages/xgboost/data.py:407, in pandas_feature_info(data, meta, feature_names, feature_types, enable_categorical)
    406 try:
--> 407     new_feature_types.append(_pandas_dtype_mapper[dtype.name])
    408 except KeyError:

KeyError: 'datetime64[ns]'

During handling of the above exception, another exception occurred:

ValueError                                Traceback (most recent call last)
File <command-6260449822151518>, line 28
     23 encoded.columns = ohe.get_feature_names_out(cat_cols)
     25 # Merge with numeric columns
     26 
     27 # Predict
---> 28 new_df["win_probability"] = model.predict_proba(new_df_encoded)[:, 1]
     29 new_df["ConvertedToSale_pred"] = (new_df["win_probability"] >= 0.5).astype(int)
     31 # Sa

In [0]:
new_df = pd.read_excel("/Volumes/hackathon/default/dataset/test_monthday.xlsx")
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   QuoteDate        1000 non-null   datetime64[ns]
 1   ProductCategory  1000 non-null   object        
 2   ProductPrice     1000 non-null   float64       
 3   QuoteAmount      1000 non-null   float64       
 4   QuoteStatus      1000 non-null   object        
 5   ConvertedToSale  1000 non-null   int64         
 6   Discount         1000 non-null   float64       
 7   Channel          1000 non-null   object        
 8   CustomerAge      999 non-null    float64       
 9   Region           999 non-null    object        
 10  PreQuoteVisits   999 non-null    float64       
 11  PaymentMethod    999 non-null    object        
 12  day              1000 non-null   object        
 13  month            1000 non-null   object        
dtypes: datetime64[ns](1), float64(5), int64(1

In [0]:
import joblib
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Load model
model = joblib.load("xgb_model.pkl")

# Load training encoder (you must have saved it earlier)
ohe = joblib.load("one_hot_encoder.pkl")  # If you didn't save it, we can rebuild with train_df

# Load new data
new_df = pd.read_excel("/Volumes/hackathon/default/dataset/test_monthday.xlsx")

# --- Drop unwanted columns if they exist ---
cols_to_drop = ["QuoteDate", "QuoteStatus", "ConvertedToSale"]  # labels/metadata
new_df = new_df.drop(columns=[c for c in cols_to_drop if c in new_df.columns], errors="ignore")

# Identify categorical columns (same as training)
cat_cols = ["ProductCategory", "Channel", "Region", "PaymentMethod", "day", "month"]  # replace with your actual categorical cols

# Apply one-hot encoding
encoded = pd.DataFrame(ohe.transform(new_df[cat_cols]), index=new_df.index)
encoded.columns = ohe.get_feature_names_out(cat_cols)

# Merge with numeric columns
new_df_encoded = pd.concat([new_df.drop(columns=cat_cols), encoded], axis=1)

# Predict
new_df["win_probability"] = model.predict_proba(new_df_encoded)[:, 1]
new_df["ConvertedToSale_pred"] = (new_df["win_probability"] >= 0.5).astype(int)

# Save predictions
new_df.to_excel("predictions.xlsx", index=False)

/Workspace/Users/mhss@udel.edu/predictions

In [0]:
# in your notebook (same place you created predictions.xlsx)
pred_path = "/dbfs/FileStore/tables/predictions/"  # folder, not a single file
import os, pandas as pd
os.makedirs(pred_path, exist_ok=True)
new_df.to_csv(pred_path + "part-0001.csv", index=False)  # write CSV

---------------------------------------------------------------------------
OSError                                   Traceback (most recent call last)
File <command-6011363768712024>, line 4
      2 pred_path = "/dbfs/FileStore/tables/predictions/"  # folder, not a single file
      3 import os, pandas as pd
----> 4 os.makedirs(pred_path, exist_ok=True)
      5 new_df.to_csv(pred_path + "part-0001.csv", index=False)

File <frozen os>:215, in makedirs(name, mode, exist_ok)

File <frozen os>:215, in makedirs(name, mode, exist_ok)

File <frozen os>:225, in makedirs(name, mode, exist_ok)

OSError: [Errno 5] Input/output error: '/dbfs/FileStore'

In [0]:
# in your notebook (same place you created predictions.xlsx)
pred_path = "/dbfs/FileStore/tables/predictions/"  # folder, not a single file
import os, pandas as pd
dbutils.fs.mkdirs(pred_path)  # create directory using dbutils
new_df.to_csv("/dbfs/FileStore/tables/predictions/part-0001.csv", index=False)  # write CSV

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-6011363768712027>, line 4
      2 pred_path = "/dbfs/FileStore/tables/predictions/"  # folder, not a single file
      3 import os, pandas as pd
----> 4 dbutils.fs.mkdirs(pred_path)  # create directory using dbutils
      5 new_df.to_csv("/dbfs/FileStore/tables/predictions/part-0001.csv", index=False)

File /databricks/python_shell/lib/dbruntime/remotefshandler/RemoteFsHandler.py:52, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
     49 class ExecutionError(Exception):
     50     pass
---> 52 raise ExecutionError(str(e)) from None

ExecutionError: (org.apache.spark.SparkSecurityException) [INSUFFICIENT_PERMISSIONS] Insufficient privileges:
User does not have permission MODIFY,SELECT on any file. SQLSTATE: 42501

JVM stacktrace:
org.apache.spark.SparkSecurityException
	at com.databricks.

In [0]:
import pandas as pd
df = pd.read_excel("/Workspace/Users/mhss@udel.edu/predictions.xlsx")
df.to_csv("predictions.csv", index=False)